# Descargar datos multiespectrales Sentinel2 y Planet NICFI

In [2]:
# Importar GEE y Iniciar sesion
import ee, geemap, os, glob, copy
import geopandas as gpd
import pandas as pd

In [3]:
ee.Initialize()

In [4]:
Map = geemap.Map(basemap='Esri.WorldImagery')
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## Importar shapefile

In [5]:
os.getcwd()

'C:\\Users\\GEOMATICA\\Curso_Geomatica\\06_BGR_Geomatica\\Datos\\Script'

In [8]:
ruta_archivos = r"C:\Users\GEOMATICA\Curso_Geomatica\06_BGR_Geomatica\Datos\Shp\Zona_buffer_500"
os.chdir(ruta_archivos) # Cambiar ruta trabajo
os.getcwd() # Consultar ruta trabajo

'C:\\Users\\GEOMATICA\\Curso_Geomatica\\06_BGR_Geomatica\\Datos\\Shp\\Zona_buffer_500'

In [9]:
lista = glob.glob("*.shp")
lista

['malla_seleccionada.shp', 'Zona_buffer_500.shp']

In [10]:
# cargar un shp
zona_gdp = gpd.read_file(lista[0])
zona_gdp

,FID,Nombre,geometry
0,0,Parcela_1,"POLYGON ((203220.135 9045443.001, 203220.135 9..."
1,0,Parcela_2,"POLYGON ((203220.135 9050443.001, 203220.135 9..."
2,0,Parcela_3,"POLYGON ((208220.135 9045443.001, 208220.135 9..."
3,0,Parcela_4,"POLYGON ((208220.135 9050443.001, 208220.135 9..."


## Convertir en Geográfico WGS84

In [11]:
# Hagamos una copia de seguridad de nuestros datos
Zona_geo = zona_gdp.copy()
# Reproject the data
Zona_wg84 = Zona_geo.to_crs(epsg=4326)

In [9]:
Zona_wg84.crs.name

'WGS 84'

## Convertir de Geopandas a GEE

In [12]:
# Crear un featureCollecion mediante GeoJSON
ee_zona_gdp = ee.FeatureCollection(Zona_wg84.__geo_interface__)

In [13]:
geometria = ee_zona_gdp.geometry()

In [14]:
# Visualizar mapa
Map.centerObject(geometria,12)
Map.addLayer(ee_zona_gdp, {"color" : "00FF11"}, "zona_gdp")
Map

Map(bottom=812.0, center=[-8.581182171995554, -77.65072954333048], controls=(WidgetControl(options=['position'…

In [15]:
# Extraer la lista 
lista_parcela = ee_zona_gdp.reduceColumns(ee.Reducer.toList(),["Nombre"]).get("list").getInfo()
print(lista_parcela)

['Parcela_1', 'Parcela_2', 'Parcela_3', 'Parcela_4']


In [16]:
Parcela = ee_zona_gdp.filterMetadata('Nombre', 'equals', lista_parcela[0])

In [17]:
os.chdir(r"D:\Descarga")

## Descargar Planet NICFI

In [18]:
# Cargar imagen Planet - NICFI
nicfi = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/americas')\
            .filter(ee.Filter.date('2023-06-01','2024-04-15'))

In [19]:
# Imprimir la lista de ID Imagenes
ID_planet = nicfi.reduceColumns(ee.Reducer.toList(),["system:index"]).get("list").getInfo()
print(ID_planet)

['planet_medres_normalized_analytic_2023-06_mosaic', 'planet_medres_normalized_analytic_2023-07_mosaic', 'planet_medres_normalized_analytic_2023-08_mosaic', 'planet_medres_normalized_analytic_2023-09_mosaic', 'planet_medres_normalized_analytic_2023-10_mosaic', 'planet_medres_normalized_analytic_2023-11_mosaic', 'planet_medres_normalized_analytic_2023-12_mosaic', 'planet_medres_normalized_analytic_2024-01_mosaic', 'planet_medres_normalized_analytic_2024-02_mosaic', 'planet_medres_normalized_analytic_2024-03_mosaic']


In [20]:
ID_planet[-1]

'planet_medres_normalized_analytic_2024-03_mosaic'

In [21]:
ID_planet[-1][-14:]+"_Planet"

'2024-03_mosaic_Planet'

In [22]:
lista_ID_planet = []

In [23]:
for r in range(len(ID_planet)):
    lista_ID_planet.append(ID_planet[r][-14:])

In [24]:
lista_ID_planet

['2023-06_mosaic',
 '2023-07_mosaic',
 '2023-08_mosaic',
 '2023-09_mosaic',
 '2023-10_mosaic',
 '2023-11_mosaic',
 '2023-12_mosaic',
 '2024-01_mosaic',
 '2024-02_mosaic',
 '2024-03_mosaic']

In [25]:
tabla_Planet = pd.DataFrame(lista_ID_planet, columns=['ID_planet'])

In [26]:
tabla_Planet.head(4)

,ID_planet
0,2023-06_mosaic
1,2023-07_mosaic
2,2023-08_mosaic
3,2023-09_mosaic


In [27]:
# Concatenar string o texto 
out_dir_planet = os.path.normpath((os.getcwd() + '/Planet' ))
out_dir_planet

'D:\\Descarga\\Planet'

In [28]:
# Concatenar string o texto 
out_dir_sentinel2 = os.path.normpath((os.getcwd() + '/Sentinel2' ))
out_dir_sentinel2

'D:\\Descarga\\Sentinel2'

In [29]:
# Crear la carpeta en el directorio
os.makedirs(out_dir_planet)
os.makedirs(out_dir_sentinel2)

In [30]:
os.chdir(out_dir_planet)

In [31]:
tabla_Planet.to_csv(out_dir_planet+"/tabla_ID_Planet.csv")

In [32]:
for i in range(len(lista_parcela)):
    Parcela = ee_zona_gdp.filterMetadata('Nombre', 'equals', lista_parcela[i])
    geometria = Parcela.geometry()
    name_parcela = lista_parcela[i]
    for k in range(len(ID_planet)):
        # Mantiene la convención de nombre de archivo original
        image = ee.Image("projects/planet-nicfi/assets/basemaps/americas" + "/" + ID_planet[k])
        # Reproyectar segun zona de estudio
        img_RS_es = image.reproject(crs="EPSG:32718", scale=4.7)
        # Recortar RS
        img_RS_es_clip = img_RS_es.clip(Parcela)
        # Descarga metodo 2
        geemap.ee_export_image(img_RS_es_clip, 
                               filename=out_dir_planet+"/"+ID_planet[k][-14:]+"_Planet_"+name_parcela+".tif", 
                               region=geometria, 
                               scale=4.77, 
                               file_per_band=False)
        print('Archivo exportado ' + ID_planet[k][-14:]+"_Planet_"+name_parcela+".tif")

Generating URL ...
Please wait ...
Data downloaded to D:\Descarga\Planet\2023-06_mosaic_Planet_Parcela_1.tif
Archivo exportado 2023-06_mosaic_Planet_Parcela_1.tif
Generating URL ...
Please wait ...
Data downloaded to D:\Descarga\Planet\2023-07_mosaic_Planet_Parcela_1.tif
Archivo exportado 2023-07_mosaic_Planet_Parcela_1.tif
Generating URL ...
Please wait ...
Data downloaded to D:\Descarga\Planet\2023-08_mosaic_Planet_Parcela_1.tif
Archivo exportado 2023-08_mosaic_Planet_Parcela_1.tif
Generating URL ...
Please wait ...
Data downloaded to D:\Descarga\Planet\2023-09_mosaic_Planet_Parcela_1.tif
Archivo exportado 2023-09_mosaic_Planet_Parcela_1.tif
Generating URL ...
Please wait ...
Data downloaded to D:\Descarga\Planet\2023-10_mosaic_Planet_Parcela_1.tif
Archivo exportado 2023-10_mosaic_Planet_Parcela_1.tif
Generating URL ...
Please wait ...
Data downloaded to D:\Descarga\Planet\2023-11_mosaic_Planet_Parcela_1.tif
Archivo exportado 2023-11_mosaic_Planet_Parcela_1.tif
Generating URL ...
Ple

## Descargar Sentinel2

In [33]:
lista_ID_S2 = []

In [34]:
os.chdir(out_dir_sentinel2)

In [35]:
for i in range(len(lista_parcela)):
    Parcela = ee_zona_gdp.filterMetadata('Nombre', 'equals', lista_parcela[i])
    geometria = Parcela.geometry()
    #Bounds = geometria.bounds()
    #Buffer = Bounds.buffer(300)
    name_parcela = lista_parcela[i]
    # Coleccion de Sentinel-2 SR
    S2_RS = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")\
                    .filterDate('2023-06-01','2024-04-14')\
                    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE','less_than',30)\
                    .filterBounds(Parcela)
    # Imprimir la lista de ID Imagenes
    ID_S2 = S2_RS.reduceColumns(ee.Reducer.toList(),["system:index"]).get("list").getInfo()
    # Gardar la lista ID
    for r in range(len(ID_S2)):
        lista_ID_S2.append(ID_S2[r])
    # Proceso de busqueda y descarga
    for k in range(len(ID_S2)):
        # Importar imagen Sentinel-2 RS 
        image = ee.Image("COPERNICUS/S2_SR_HARMONIZED" + "/" + ID_S2[k])
        # Seleccionar iamgenes por bandas
        S2_img_RS = image.select(["B1","B2","B3","B4","B5","B6","B7","B8","B9","B11","B12"])
        # Recortar segun zona estudio
        S2_img_RS_clip = S2_img_RS.clip(Parcela)      
        # Descarga 
        geemap.ee_export_image(S2_img_RS_clip, 
                               filename=out_dir_sentinel2+"/"+ID_S2[k][0:16]+name_parcela+".tif", 
                               region=geometria, 
                               scale=10, 
                               file_per_band=False)
        print('Archivo exportado ' + ID_S2[k][0:16]+name_parcela+".tif")


Generating URL ...
Please wait ...
Data downloaded to D:\Descarga\Sentinel2\20230602T152641_Parcela_1.tif
Archivo exportado 20230602T152641_Parcela_1.tif
Generating URL ...
Please wait ...
Data downloaded to D:\Descarga\Sentinel2\20230602T152641_Parcela_1.tif
Archivo exportado 20230602T152641_Parcela_1.tif
Generating URL ...
Please wait ...
Data downloaded to D:\Descarga\Sentinel2\20230617T152639_Parcela_1.tif
Archivo exportado 20230617T152639_Parcela_1.tif
Generating URL ...
Please wait ...
Data downloaded to D:\Descarga\Sentinel2\20230622T152641_Parcela_1.tif
Archivo exportado 20230622T152641_Parcela_1.tif
Generating URL ...
Please wait ...
Data downloaded to D:\Descarga\Sentinel2\20230622T152641_Parcela_1.tif
Archivo exportado 20230622T152641_Parcela_1.tif
Generating URL ...
Please wait ...
Data downloaded to D:\Descarga\Sentinel2\20230627T152639_Parcela_1.tif
Archivo exportado 20230627T152639_Parcela_1.tif
Generating URL ...
Please wait ...
Data downloaded to D:\Descarga\Sentinel2\

In [37]:
lista_ID = []

In [38]:
for f in range(len(lista_ID_S2)):
    dato = lista_ID_S2[f][0:15]
    lista_ID.append(dato)

In [39]:
len(lista_ID)

128

In [40]:
# Extraer valores únicos como un conjunto
valores_unicos = set(lista_ID)

In [41]:
len(valores_unicos)

19

In [42]:
tabla_sentinel2 = pd.DataFrame(valores_unicos, columns = ["ID_RS"])

In [43]:
tabla_sentinel2.head(4)

,ID_RS
0,20230707T152649
1,20230910T152641
2,20230816T152649
3,20230925T152639


In [44]:
tabla_sentinel2.to_csv(out_dir_sentinel2+"/"+"tabla_sentinel2_ID_RS.csv")